In [1]:
import pandas as pd
#from nltk.tokenize import RegexpTokenizer
#from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

C:\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
   
#df = pd.read_csv('data/grill-and-bbq-normalize.csv', encoding='utf-8', header=1, sep='|')
dfrecipe = pd.read_csv('data/recipeset.csv', encoding='utf-8', header=0, sep='|')
dfingre = pd.read_csv('data/ingredients.csv', encoding='utf-8', header=0, sep='|')

df = []
for i in range(len(dfrecipe)):
    j = [dfingre[dfingre['dish_id'] == dfrecipe.dish_id[i]]]
    str = j[0]['ingredient'].values 
    df.append([str[k] for k in range(len(str))])

#recipeSet = [row[5].lower() for row in df.itertuples()]

#bow_ingre = [reci.split(" ") for reci in recipeSet]
bow_ingre = df

phrases = Phrases(bow_ingre, min_count=2, threshold=2)
bigram = Phraser(phrases)
documents = [bigram[j] for j in bow_ingre]

new_bowingre = [doc for doc in documents]
corp_dict = corpora.Dictionary(new_bowingre)
    
corpus = [corp_dict.doc2bow(ingre) for ingre in new_bowingre]

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = corp_dict, passes=20)
#print(ldamodel.print_topics(num_topics=3, num_words=4))
#print(ldamodel.print_topics(num_topics=3, num_words=5))
#print(ldamodel.print_topics(num_topics=3, num_words=6))
print(ldamodel.print_topics(num_topics=20, num_words=7))



[(0, '0.190*"steak" + 0.127*"kosher salt" + 0.106*"black pepper" + 0.080*"garlic" + 0.058*"olive oil" + 0.050*"oregano" + 0.037*"extra virgin olive oil"'), (1, '0.139*"chicken" + 0.079*"vegetable oil" + 0.066*"garlic" + 0.063*"honey" + 0.047*"pepper" + 0.040*"soy sauce" + 0.036*"white wine"'), (2, '0.129*"season" + 0.090*"pepper" + 0.082*"garlic" + 0.067*"beef" + 0.055*"coriander" + 0.054*"salt" + 0.047*"lemon"'), (3, '0.109*"onion" + 0.103*"worcestershire sauce" + 0.085*"mustard" + 0.082*"pepper" + 0.079*"ketchup" + 0.051*"brown sugar" + 0.048*"vinegar"'), (4, '0.067*"chili" + 0.064*"mint" + 0.062*"cloves" + 0.045*"scallion" + 0.045*"ginger root" + 0.042*"rice" + 0.041*"short ribs"'), (5, '0.210*"butter" + 0.063*"parsley" + 0.058*"pepper" + 0.035*"chicken" + 0.034*"cream" + 0.034*"salt" + 0.031*"cheddar cheese"'), (6, '0.097*"cilantro" + 0.073*"lime juice" + 0.059*"garlic" + 0.052*"salt" + 0.047*"pepper" + 0.045*"olive oil" + 0.039*"cumin"'), (7, '0.173*"lemon" + 0.125*"chicken breast

TO VERIFY IF RECIPE IS NEW

In [7]:
import numpy as np
from pandas import Series

s = pd.Series([k for k in bow_ingre])
for k in s:
    ind=[]
    ky = Series(k)
    #This should return True on all ingredients - Passed
    #ind=ky.isin(['onion', 'red pepper flakes','pork', 'toast', 'coffee', \
    #             'worcestershire sauce','black pepper' ,'liquid smoke','garlic'])
    #test2.7 - Failed
    #ind=ky.isin(['olive oil', 'garlic','lemon', 'salt', 'black pepper','chicken breast','pepper'])
    #test1.7 - Passed (this is soy-ginger chicken without olive oil but added cilantro)
    ind=ky.isin(['garlic', 'soy sauce','ginger', 'cilantro', 'lime juice','sugar', 'chicken'])
    #test0.7 remove black pepper since same with pepper - Passed
    #(this is similar to korean baked pork belly or pork tenderloin)
    #ind=ky.isin(['salt','onion','garlic','brown sugar','pepper','pork']) 
    #test2.6 - same with 2.7
    #test1.6 - Passed but what is meat?
    #ind=ky.isin(['garlic', 'soy sauce','ginger', 'cilantro', 'lime juice'])
    #test0.6 - Passed but what is meat?
    #ind=ky.isin(['salt','onion','garlic','brown sugar','pepper'])
    if (np.sum(ind)==len(ind)):
        print('Found the recipe with ' + ky)



0    Found the recipe with chicken
dtype: object
0    Found the recipe with garlic
dtype: object
0     Found the recipe with soy sauce
1    Found the recipe with lime juice
dtype: object
0    Found the recipe with garlic
dtype: object
0    Found the recipe with chicken
dtype: object
0    Found the recipe with ginger
dtype: object
0    Found the recipe with garlic
dtype: object
0       Found the recipe with garlic
1    Found the recipe with soy sauce
dtype: object


In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(ldamodel, corpus, corp_dict)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     11.162058        1       1 -0.087290  0.004747
6       9.501208        1       2  0.057968  0.039964
10      8.450059        1       3 -0.146523  0.207860
3       7.502537        1       4 -0.160207 -0.120259
12      6.984496        1       5  0.211130 -0.010215
7       5.853424        1       6  0.230314 -0.038421
1       5.343422        1       7 -0.006502  0.094764
19      5.325450        1       8  0.148367 -0.044375
0       4.904363        1       9  0.223308  0.066377
13      4.403543        1      10 -0.052935  0.081560
9       3.951030        1      11  0.145745  0.044637
5       3.844224        1      12  0.044760 -0.150054
15      3.624728        1      13 -0.065732  0.275984
2       3.512662        1      14 -0.034702  0.008261
14      3.199453        1      15 -0.144943 -0.077518
4       3.057829        1      16 -0.130515  0.145858
11      2.958485        1      17 -0.033666 -0.114837
16      2.616134        1      18 -0.244904 -0.150431
18      2.037652        1      19 -0.058436 -0.186077
8       1.767243        1      20  0.104764 -0.077825, topic_info=     Category         Freq                        Term        Total  loglift  \
term                                                                           
1     Default   653.000000                       lemon   653.000000  30.0000   
313   Default   295.000000                       steak   295.000000  29.0000   
114   Default   269.000000                      butter   269.000000  28.0000   
178   Default   811.000000                   olive oil   811.000000  27.0000   
358   Default   509.000000                   soy sauce   509.000000  26.0000   
597   Default   307.000000                     paprika   307.000000  25.0000   
385   Default   356.000000                      ginger   356.000000  24.0000   
392   Default   348.000000                 kosher salt   348.000000  23.0000   
449   Default   322.000000                        pork   322.000000  22.0000   
261   Default   495.000000                     chicken   495.000000  21.0000   
460   Default   307.000000                    cilantro   307.000000  20.0000   
21    Default   282.000000        worcestershire sauce   282.000000  19.0000   
602   Default   902.000000                black pepper   902.000000  18.0000   
456   Default   572.000000              chicken breast   572.000000  17.0000   
482   Default   854.000000                      pepper   854.000000  16.0000   
28    Default   285.000000                     mustard   285.000000  15.0000   
500   Default   251.000000                  lime juice   251.000000  14.0000   
510   Default   751.000000                       onion   751.000000  13.0000   
409   Default   144.000000                      season   144.000000  12.0000   
8     Default   119.000000              barbecue sauce   119.000000  11.0000   
191   Default   543.000000                 brown sugar   543.000000  10.0000   
599   Default   110.000000                         bbq   110.000000   9.0000   
440   Default   221.000000                     ketchup   221.000000   8.0000   
330   Default  1102.000000                        salt  1102.000000   7.0000   
505   Default   362.000000                       sugar   362.000000   6.0000   
391   Default   148.000000                    rosemary   148.000000   5.0000   
384   Default   229.000000                     cayenne   229.000000   4.0000   
558   Default   250.000000                       cumin   250.000000   3.0000   
441   Default   117.000000                   pineapple   117.000000   2.0000   
514   Default   207.000000                     parsley   207.000000   1.0000   
...       ...          ...                         ...          ...      ...   
615   Topic20     7.979151                        feta     8.913784   3.9250   
435   Topic20     7.231953                       